In [2]:
#making final schedules
import pandas as pd
from datetime import datetime, timedelta

df = pd.read_csv('../Data/dataset_for_schedules/traintripper_trainschedules.csv')

durations = []
for train_number in df['train_number'].unique():
    relevant_data = df[df['train_number'] == train_number]
    duration = 0
    prev_day_count = relevant_data.iloc[0]['day_count']

    for i in range(len(relevant_data)):
        if i==0:
            duration = 0
            durations.append(duration)
            departure = datetime.strptime(str(relevant_data.iloc[i]['departure_time']), '%H:%M:%S')
            continue
        else:
            arrival = datetime.strptime(str(relevant_data.iloc[i]['arrival_time']), '%H:%M:%S')
            if relevant_data.iloc[i]['day_count'] > prev_day_count:
                day = int(relevant_data.iloc[i]['day_count']) - int(prev_day_count)
                arrival += timedelta(days=day) # Add a day to the arrival time
            diff = arrival - departure
            duration = diff.total_seconds()/60 

            durations.append(duration)
        

df['duration_minutes'] = durations


# to csv
df.to_csv('../Data/processed_data/final_schedules.csv', index=False)
df

FileNotFoundError: [Errno 2] No such file or directory: '../Data/dataset_for_schedules/traintripper_trainschedules.csv'

In [1]:
# one hot encoding
import pandas as pd

df2 = pd.read_csv('../Data/processed_data/all_fare_analyzed.csv')

dummies = pd.get_dummies(df2['classCode'])
merged = pd.concat([df2, dummies], axis='columns')
final = merged.drop(['classCode'], axis='columns')
final

,trainNumber,fromStnCode,toStnCode,if_offering_catering,if_dynamic_fare,fare_to_predict,1A,2A,2S,3A,CC,SL
0,1027,NK,CKTD,False,False,2470,False,True,False,False,False,False
1,1027,NK,CKTD,False,False,2470,False,True,False,False,False,False
2,1027,NK,CKTD,False,False,2470,False,True,False,False,False,False
3,1027,NK,CKTD,False,False,2470,False,True,False,False,False,False
4,1027,NK,CKTD,False,False,2470,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
324327,22645,ET,ERN,False,False,2010,False,False,False,True,False,False
324328,22645,ET,ERN,False,False,755,False,False,False,False,False,True
324329,22645,ET,ERS,False,False,2935,False,True,False,False,False,False
324330,22645,ET,ERS,False,False,2010,False,False,False,True,False,False


In [2]:
final.to_csv("../Data/processed_data/all_hot_encoded.csv", index=False)

In [4]:
import pandas as pd

df = pd.read_csv('../Data/processed_data/final_schedules.csv')

distance_map = {}

for index, row in df.iterrows():
    key = (row['train_number'], row['station_code'] )
    distance = row['distance']
    duration = row['duration_minutes']
    # distance_map[key] = distance
    distance_map[key] = (distance, duration)

static_df = pd.read_csv('../Data/processed_data/all_hot_encoded.csv')

# Create a new 'distance' column and initialize it with None
static_df['distance'] = None
static_df['duration'] = None

# Iterate through the rows of the static DataFrame and populate the 'distance' column
for index, row in static_df.iterrows():
    train_number = row['trainNumber']
    from_stn = row['fromStnCode']
    to_stn = row['toStnCode']
    
    # Look up the distance in the distance_map using trainNumber and fromStn
    key1 = distance_map.get((train_number, from_stn), None)
    key2 = distance_map.get((train_number, to_stn), None)
    
    cal_distance = key2[0] - key1[0]
    cal_duration = key2[1] - key1[1]

    # Update the 'distance' column with the distance value
    static_df.at[index, 'distance'] = cal_distance
    static_df.at[index, 'duration'] = cal_duration    

In [5]:
map1 = {str(k): v for k, v in distance_map.items()}

In [6]:
import json
json.dump(map1, open("../Data/precomputes/distance_map", 'w'))

In [6]:
static_df

,trainNumber,fromStnCode,toStnCode,totalFare,1A,2A,2S,3A,CC,SL,distance,duration
0,11464,JBP,SRID,1175,True,False,False,False,False,False,54,33.0
1,11464,JBP,SRID,710,False,True,False,False,False,False,54,33.0
2,11464,JBP,SRID,505,False,False,False,True,False,False,54,33.0
3,11464,JBP,SRID,145,False,False,False,False,False,True,54,33.0
4,11464,JBP,KKB,1175,True,False,False,False,False,False,69,49.0
...,...,...,...,...,...,...,...,...,...,...,...,...
326638,22420,BE,ARJ,915,False,False,False,True,False,False,532,570.0
326639,22420,BE,ARJ,915,False,False,False,True,False,False,532,570.0
326640,22420,BE,ARJ,915,False,False,False,True,False,False,532,570.0
326641,22420,BE,ARJ,915,False,False,False,True,False,False,532,570.0


In [7]:
static_df.drop(['trainNumber', 'fromStnCode', 'toStnCode'], axis=1, inplace=True)

In [8]:
static_df.to_csv('../Data/processed_data/all_final_for_model.csv', index=False)